In [ ]:
'''
STILL A TODO.  Add in right SQL script.
'''

In [1]:
import pandas as pd
import tqdm
import numpy as np
from collections import OrderedDict
from collections import defaultdict
import pickle
import glob
from matplotlib import pyplot as plt
%matplotlib inline

import plotly
import plotly.graph_objects as go

In [2]:
import pandas.io.sql
import pyodbc
import tqdm

import pickle
import glob

In [5]:
import sys
sys.path.append('C:\\Users\\tonys\\Documents\\Research\\characterizationPaperPackage\\inst\\py\\')

In [6]:
from settings import *

Imported settings...
Current path:  C:\Users\tonys\Documents\Research\characterizationPaperPackage\inst\py\


In [7]:
# Create the connection
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+db+';UID='+user+';PWD='+ password)

In [8]:
sql_query_string = 'select * from ' + db + '.results.pbr_sexdiff_cohort_covarate_summary_v5'
all_conds_df = pandas.io.sql.read_sql(sql_query_string, conn)

In [9]:
sys.getsizeof(all_conds_df)

333813930

In [10]:
all_conds_df.head()

,source_name,cohort_definition_id,cohort_definition_name,concept_id,concept_name,num_persons_w_covariate,num_persons_total,pct_persons_w_covariate,num_females,num_males,pct_females,prev_females,prev_males,rr_female
0,ohdsi_cumc_2021q1r2,11232,[PL 44784217001] Cardiac arrhythmia referent c...,27918,Eosinophilic esophagitis,47,86868,0.000541,17,30,0.361702,0.038752,0.069769,0.555429
1,ohdsi_cumc_2021q1r2,11125,[PL 134442005] Systemic sclerosis or or locali...,27918,Eosinophilic esophagitis,4,1966,0.002035,2,2,0.500000,0.119832,0.673401,0.177951
2,ohdsi_cumc_2021q1r2,11189,[PL 434592003] B-cell lymphoma incident cohort...,27918,Eosinophilic esophagitis,3,2413,0.001243,2,1,0.666667,0.167364,0.082102,2.038494
3,ohdsi_cumc_2021q1r2,11114,[PL 31967003] Nausea Events All Events of refe...,28060,Streptococcal sore throat,857,49704,0.017242,569,288,0.663944,1.657105,1.874146,0.884192
4,ohdsi_cumc_2021q1r2,11225,[PL 4281232001] Type B viral hepatitis referen...,28060,Streptococcal sore throat,9,2186,0.004117,7,2,0.777778,0.691700,0.170358,4.060277


In [11]:
# New criteria
DQD = pd.read_csv('../csv/OMOP_dqd.csv')

# Minumum number of humans
num_persons = all_conds_df['num_persons'] >= 50

# Remove OHDSI's DQD list of sex-specific
not_sex_specific = ~all_conds_df['concept_id'].isin(DQD[~DQD['plausibleGender'].isnull()]['conceptId'])

# Limit diseases that are super prevalent in only females
not_female_only = all_conds_df['pct_female'] < 99
not_male_only   = all_conds_df['pct_female'] > 1

# Remove conditions that have male/female in their names
sex_specific_name = []
for cond in all_conds_df['concept_name']:
    cond = cond.lower()
    if 'male' in cond or 'maternal' in cond or 'uterine' in cond \
    or 'pregnancy' in cond or 'menopausal' in cond or 'uterus' in cond \
    or 'umbilical' in cond or 'gestation' in cond or 'cervix' in cond \
    or 'birth' in cond or 'vaginal' in cond or 'vagina' in cond or 'vulva' in cond or 'placenta' in cond \
    or 'ovary' in cond or 'miscarriage' in cond or 'postpartum' in cond or 'live born' in cond or 'obstet' in cond:
        # print(cond)
        sex_specific_name.append(False)
    elif 'penis' in cond or 'testis' in cond or 'testicle' in cond:
        sex_specific_name.append(False)
    else:
        sex_specific_name.append(True)
        
not_a_legit_concept = all_conds_df['concept_name'] == 'No matching concept'

valid_conds = (num_persons) & (not_sex_specific) & (sex_specific_name) & (not_male_only) & (not_female_only) & (~not_a_legit_concept)

KeyError: 'num_persons'